In [1]:
%pwd

'c:\\Users\\Asus_M\\Desktop\\accounting-chatbot\\notebooks'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Asus_M\\Desktop\\accounting-chatbot'

In [11]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

#### Extract the data from the PDF:

In [12]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls = PyPDFLoader)
    documents = loader.load()
    return documents

In [13]:
extracted_data = load_pdf_file(data = "data/")

#### Split the data into Text chunks:

In [14]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size =500,
        chunk_overlap = 20
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [15]:
text_chunks = text_split(extracted_data)
print("Length odf text chunks :",len(text_chunks))

Length odf text chunks : 2285


#### Download Embedding Model from Hugging Face Hub:

In [16]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name = "sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

##### Testing the enbedding model :

In [18]:
query_result = embeddings.embed_query("Hello world!")
print("length :",len(query_result))

length : 384


In [19]:
load_dotenv()
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

In [20]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [21]:
from pinecone import ServerlessSpec
from pinecone.grpc  import  PineconeGRPC as Pinecone

pc = Pinecone(api_key = PINECONE_API_KEY)

index_name = "comptachatbot"
pc.create_index(
    name=index_name,
    dimension= 384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

##### Embed each chunk and upsert the embeddings into Pinecone index:

In [22]:
docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name=index_name,
    embedding=embeddings
    )

##### Loading the existing Pinecone index:

In [23]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
    )

In [24]:
retriever = docsearch.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" :3}
)

In [26]:
retrieved_docs = retriever.invoke("Expliquer LES COÛTS MARGINAUX ET LES COÛTS DIFFERENTIELS")

In [27]:
retrieved_docs

[Document(id='ffd73305-580c-40aa-a46b-db77d6b043a4', metadata={'page': 50.0, 'source': 'data\\6606030.pdf'}, page_content='Il n’est guère possible de relier les différents coûts par tiels à une typologie simple des entreprises ou des marchés \nsur lesquels elles opèrent. C’est pourquoi nous présente rons successivement les principales familles de coûts \npartiels en mentionnant à chaque fois les différents problèmes qu’elles sont le plus aptes à résoudre. \n2 Les coûts partiels : fondements techniques et \norganisationnels'),
 Document(id='25f35fd1-5bc8-481a-8400-a2094ba0470a', metadata={'page': 62.0, 'source': 'data\\6606030.pdf'}, page_content='coût de la voiture supplémentaire n’est pas considéré ici comme variable par rapport au nombre de passagers mais par rapport à la seule distance. Le coût de l’accroisse ment de capacité fait donc partie du coût marginal et a \nfortiori du coût différentiel, alors qu’il ne fait partie du coût variable que s’il entraîne des coûts variables \nsup

In [28]:
llm = ChatGroq(model="llama-3.1-70b-versatile",
               temperature=0,
               max_tokens=500)

In [29]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [30]:
system_prompt = ("Vous êtes un assistant pour une tâche de réponse à des questions. "
                 "Utilisez le contexte suivant pour répondre à la question. "
                 "Si vous ne connaissez pas la réponse, dites que vous ne savez pas. "
                 "Utilisez au maximum trois phrases et gardez la réponse concise. "
                 "\n\n"
                 "{context}"
                 )
prompt = ChatPromptTemplate(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [31]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [32]:
response = rag_chain.invoke({"input":"Expliquer LES COÛTS MARGINAUX ET LES COÛTS DIFFERENTIELS"})
print(response["answer"])

Les coûts marginaux et les coûts différentiels sont deux concepts liés mais distincts en comptabilité et en économie.

Le coût marginal est le coût supplémentaire engagé pour produire une unité supplémentaire d'un bien ou d'un service. Il représente la variation du coût total lorsque la production augmente d'une unité.

Le coût différentiel, quant à lui, est la variation du coût total lorsque la production augmente d'une certaine quantité, mais pas nécessairement d'une unité. Il représente la différence entre le coût total de la production d'une certaine quantité et le coût total de la production d'une quantité inférieure.

En d'autres termes, le coût marginal est une mesure du coût supplémentaire pour une unité supplémentaire, tandis que le coût différentiel est une mesure du coût supplémentaire pour une quantité supplémentaire.
